In [42]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys


In [43]:
image1 = Image.open("paris/paris_a.jpg")
image2 = Image.open("paris/paris_b.jpg")
img1 = np.asarray(image1)
img2 = np.asarray(image2)
#plt.imshow(image)
#points = plt.ginput(5, show_clicks=True)

In [34]:
with open('paris1-2_10points.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)

In [38]:
def computeH(points_im1, points_im2):
    matrixA = []
    for i in range(len(points_im1)):
        xs, ys = points_im1[i][0], points_im1[i][1]
        xd, yd = points_im2[i][0], points_im1[i][1]
        two_nineMatrix = np.array([[xs, ys, 1,0,0,0, -xd*xs, -xd*ys, -xd], [0,0,0,xs,ys,1,-yd*xs,-yd*ys,-yd]])
        matrixA.append(two_nineMatrix[0])
        matrixA.append(two_nineMatrix[1])
    return matrixA

In [39]:
A = computeH(a,b)

In [40]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [44]:
homography = (-1)*vh[-1].reshape(3,3)

In [73]:
def warp(image,homography):
    t_inverse = np.linalg.inv(homography)
    boundries = np.matmul(homography, np.array([[0,0,640,640],[0,480,0,480],[1,1,1,1]]))
    boundriesDenormalized = np.matmul(boundries, np.diag(1/boundries[-1]))
    print(boundriesDenormalized, np.shape(boundriesDenormalized))
    newPositions = []
    wrapped = np.zeros(np.shape(image))
    for i in range(len(wrapped)-1):
        for j in range(len(wrapped[i])-1):
            coordinates = np.matmul(t_inverse, np.array([[i],[j],[1]]))
            coordinates = (coordinates* (1/coordinates[2][0]))[0:2]
            if coordinates[0] * coordinates[1] > 0 and int(coordinates[0]) < len(image)-1 and int(coordinates[1]) < len(image[0])-1:
                leftTopX, leftTopY, leftTopVal = int(coordinates[0]), int(coordinates[1]), image[int(coordinates[0])][int(coordinates[1])]
                rightTopX, rightTopY, rightTopVal = int(coordinates[0]) +1 , int(coordinates[1]), image[int(coordinates[0])+1][int(coordinates[1])]
                leftBottomX, leftBottomY, leftBottomVal = int(coordinates[0]), int(coordinates[1])+1, image[int(coordinates[0])][int(coordinates[1])+1]
                rightBottomX, rightBottomY, rightBottomVal = int(coordinates[0])+1, int(coordinates[1])+1, image[int(coordinates[0])+1][int(coordinates[1])+1]
                a=coordinates[0]%1
                b= int(coordinates[1])+1 - coordinates[1]
                bilinearInterpolation = leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b
                wrapped[i][j] = bilinearInterpolation
    return np.asarray(wrapped)

In [71]:
def fwarp(image,homography):
    wrapped = np.zeros(np.shape(image))
    newPositions = []
    for i in range(len(image)):
        for j in range(len(image[i])):
            coordinates = np.matmul(homography, np.array([[i],[j],[1]]))
            newPositions.append(coordinates* (1/coordinates[2][0]))
    return np.asarray(newPositions).reshape(np.shape(np.asarray(image)))[:,:,0:2]

In [74]:
back_res = warp(np.asarray(image),homography)

[[-5.39435161e+02 -1.50111322e+03  2.88670204e+02  3.59702130e+02]
 [-1.36127890e+02  8.77198940e+02  1.27774306e+02  4.55817415e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]] (3, 4)


In [20]:
res = fwarp(np.asarray(image),hstar)

In [76]:
np.shape(back_res)

(480, 640, 3)

In [78]:
imgResult = Image.fromarray(back_res.astype(np.uint8), 'RGB')
imgResult.save("warped.jpg")

In [22]:
def construct(image,wrap):
    res = np.round(wrap)
    x_min = np.amin(res[:,:,0])
    y_min = np.amin(res[:,:,1])
    res[:,:,0] = res[:,:,0] + np.abs(x_min)
    res[:,:,1] = res[:,:,1] + np.abs(y_min)
    x_max = np.amax(res[:,:,0]).astype(int)
    y_max = np.amax(res[:,:,1]).astype(int)
    res = res.astype(int)
    print(x_max,y_max)
    scale_X = (len(image) / x_max) 
    scale_Y = (len(image[0]) / y_max)
    projected = np.zeros(np.shape(image))
    for i in range(len(image)):
        for j in range(len(image[i])):
            c = res[i][j]
            #print(int(c[0] * scale_X), int(c[1] * scale_Y) )
            projected[int(c[0] * scale_X)-1][int(c[1] * scale_Y)-1] = image[i][j]
    return projected

In [23]:
imageMatrix=construct(im,res)
imgResult = Image.fromarray(imageMatrix.astype(np.uint8), 'RGB')
imgResult.save("10transform.jpg")

3388 2739


In [24]:
print(np.amin(res[:,:,0]),np.amax(res[:,:,0]))
print(np.amin(res[:,:,1]),np.amax(res[:,:,1]))

-3138.7226735397476 248.77620614530588
-136.12788992140727 2602.759349287685


In [51]:
np.shape(np.array([[1],[2],[3]]))

(3, 1)

In [61]:
int(4.6)

4